In [1]:
%%capture

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
fileId = drive.CreateFile({'id': '1dGB9Lp70ania8L8DwXSI77YzwKsMuIBq'}) 
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import pandas as pd
from pathlib import Path
from IPython.display import display 
from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout

In [ ]:
!unzip temp.zip -d ./

In [6]:
df = pd.read_csv('train.truth.csv')

In [7]:
df.head()

,fn,label
0,0-10049200_1891-09-16_1958.jpg,rotated_left
1,0-10110600_1985-09-17_2012.jpg,rotated_left
2,0-10126400_1964-07-07_2010.jpg,upright
3,0-1013900_1917-10-15_1960.jpg,rotated_right
4,0-10166400_1960-03-12_2008.jpg,upside_down


In [8]:
import glob
images = []
labels = []
total = 0
for img_path in glob.glob('train/*.jpg'):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    images.append(img_array)
    path = img_path.split('/')[1]
    labels.append(df.loc[df['fn'] == path, 'label'].iloc[0])
    total += 1

In [9]:
print(len(images))
print(len(labels ))
print('There are' ,total,"images")

48896
48896
There are 48896 images


In [10]:
data = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

In [11]:
data.shape

(48896, 64, 64, 3)

In [12]:
labels.shape

(48896,)

In [13]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [14]:
input_shape = data[0].shape

In [15]:
input_shape

(64, 64, 3)

In [16]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape, padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3, 3),padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(256, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(256, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3),  padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation("softmax"))

In [17]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 128
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS), metrics=["accuracy"])

In [18]:
H = model.fit(x=data, y=labels, epochs=EPOCHS, verbose=1)

Epoch 1/10
1528/1528 [==============================] - 190s 124ms/step - loss: 1.8112 - accuracy: 0.4149
Epoch 2/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.6464 - accuracy: 0.8187
Epoch 3/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.1558 - accuracy: 0.9485
Epoch 4/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.1094 - accuracy: 0.9645
Epoch 5/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.0901 - accuracy: 0.9709
Epoch 6/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.1020 - accuracy: 0.9678
Epoch 7/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.0741 - accuracy: 0.9765
Epoch 8/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.0556 - accuracy: 0.9816
Epoch 9/10
1528/1528 [==============================] - 191s 125ms/step - loss: 0.0540 - accuracy: 0.9825
Epoch 10/10
1528/1528 [=======================

In [19]:
fileTest = drive.CreateFile({'id': '1fWXqatTYVgeITF6ozAeMY9pePyUTOJW5'}) 
fileTest.GetContentFile('test.zip')  # Save Drive file as a local file

In [ ]:
!unzip test.zip -d ./

In [40]:
import glob
images = []
image_path = []
total = 0
for img_path in glob.glob('test/*.jpg'):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    images.append(img_array)
    path = img_path.split('/')[1]
    image_path.append(path)
    total += 1

In [41]:
print('There are' ,total,"test images")

There are 5361 test images


In [42]:
data = np.array(images, dtype="float") / 255.0

In [43]:
preds = model.predict(data)

In [44]:
y_test_original = lb.inverse_transform(preds)

In [45]:
y_test_original.shape

(5361,)

In [46]:
y_test_original[0]

'rotated_left'

In [47]:
ans = pd.DataFrame({'fn':image_path, 'label':y_test_original})

In [48]:
ans.head()

,fn,label
0,91-30104091_1993-02-16_2012.jpg,rotated_left
1,94-12758794_1970-09-19_2009.jpg,rotated_right
2,93-3023093_1951-12-03_2006.jpg,rotated_left
3,93-9589293_1992-05-26_2008.jpg,upright
4,94-6339494_1971-04-21_2008.jpg,rotated_right


In [50]:
ans.to_csv("test.preds.csv", encoding='utf-8', index=False)